# 0. Install Dependencies

!pip install tensorflow==2.3.1 gym keras-rl2 gym[atari]

In [ ]:
!pip install gym gym[atari] keras-rl2 atari-py

     |████████████████████████████████| 52 kB 662 kB/s 
     |████████████████████████████████| 462 kB 9.3 MB/s 


In [ ]:
# Habilitar o acesso ao disco do Google Drive para leitura e escrita.
# 
from google.colab import drive
#drive.mount('drive')
# Mount drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
# Importando a ROM gravada no Google Drive
!python -m atari_py.import_roms '/drive/My Drive/SpaceInvaders_ROMS/'

copying space_invaders.bin from /drive/My Drive/SpaceInvaders_ROMS/Space Invaders (1980) (Atari, Richard Maurer - Sears) (CX2632 - 49-75153) ~.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/space_invaders.bin


In [ ]:
# Verificando as ROMs da Atari disponíveis.
import atari_py
atari_py.list_games()

['tetris', 'space_invaders']

# 1. Test Random Environment with OpenAI Gym

In [ ]:
import gym 
import random

In [ ]:
# Carregando o ambiente para treino
env = gym.make('SpaceInvaders-v0')

SpaceInvaders-v0

Maximize your score in the Atari 2600 game SpaceInvaders.   
In this environment, the observation is an RGB image of the screen, which is an array of shape (210, 160, 3).  
Each action is repeatedly performed for a duration of kkk frames, where kkk is uniformly sampled from \{2, 3, 4\}.


In [ ]:
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [ ]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [ ]:
## Esta rotina abre a janela do jogo e joga com comandos aleatórios. 
# episodes = 8
# for episode in range(1, episodes+1):
#     state = env.reset()
#     done = False
#     score = 0 
    
#     while not done:
#         env.render()
#         action = random.choice([0,1,2,3,4,5])
#         n_state, reward, done, info = env.step(action)
#         score+=reward
#     print('Episode:{} Score:{}'.format(episode, score))
# env.close()

# 2. Create a Deep Learning Model with Keras

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [ ]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model


# model.add(
# Convolution2D - Metodo para procurar diferenças nas imagens
#(32 - Numero de filtros para detectar diferenças nas imagens
#(8,8) - Tamanho da matriz em que o filtro será aplicado, 
# strides=(4,4) - Tamanho do passo em pixeis que a matriz filtro se desloca na imagem.
# activation='relu', 
# input_shape=(3,height, width, channels)))


In [ ]:
model = build_model(height, width, channels, actions)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 3, 51, 39, 32)     6176      
                                                                 
 conv2d_1 (Conv2D)           (None, 3, 24, 18, 64)     32832     
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 22, 16, 64)     36928     
                                                                 
 flatten (Flatten)           (None, 67584)             0         
                                                                 
 dense (Dense)               (None, 512)               34603520  
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 6)                 1

# 3. Build Agent with Keras-RL

In [ ]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [ ]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [ ]:
# Modelo foi removido como solução ao problema de treino. 
del model

In [ ]:
model = build_model(height, width, channels, actions)

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3))

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
import time

#Tic
sys_start_time = time.perf_counter()

In [ ]:
dqn.fit(env, nb_steps=5000, visualize=False, verbose=2)

Training for 5000 steps ...


/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


  697/5000: episode: 1, duration: 19.996s, episode steps: 697, steps per second:  35, episode reward: 110.000, mean reward:  0.158 [ 0.000, 30.000], mean action: 2.433 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --


/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


 1427/5000: episode: 2, duration: 416.065s, episode steps: 730, steps per second:   2, episode reward: 185.000, mean reward:  0.253 [ 0.000, 30.000], mean action: 2.516 [0.000, 5.000],  loss: 355.002717, mean_q: -1.029197, mean_eps: 0.890785
 2064/5000: episode: 3, duration: 607.559s, episode steps: 637, steps per second:   1, episode reward: 120.000, mean reward:  0.188 [ 0.000, 30.000], mean action: 2.457 [0.000, 5.000],  loss: 0.394675, mean_q: -2.931919, mean_eps: 0.842950
 2795/5000: episode: 4, duration: 717.077s, episode steps: 731, steps per second:   1, episode reward: 225.000, mean reward:  0.308 [ 0.000, 30.000], mean action: 2.461 [0.000, 5.000],  loss: 0.909446, mean_q: -3.105091, mean_eps: 0.781390
 3495/5000: episode: 5, duration: 682.278s, episode steps: 700, steps per second:   1, episode reward: 85.000, mean reward:  0.121 [ 0.000, 20.000], mean action: 2.377 [0.000, 5.000],  loss: 0.274312, mean_q: -3.039830, mean_eps: 0.716995
 4074/5000: episode: 6, duration: 565.4

In [ ]:
#Toc
delta_t = time.perf_counter() - sys_start_time

# Exibindo o tempo utilizado para o processo.
print('\n \n')
if delta_t > 3600:
    delta_t = delta_t / 3600
    print("Elapsed time: %.1f hours" % ((delta_t)))
elif delta_t > 60 and delta_t <= 3600:
    delta_t = delta_t / 60
    print("Elapsed time: %.1f min" % ((delta_t)))
else:
    print("Elapsed time: %.1f s" % ((delta_t)))


 

Elapsed time: 1.1 hours


# 4. Saving Agent to Disk

In [ ]:
dqn.save_weights('/drive/My Drive/RL_weights/SpaceInvaders_5000.h5f')